Carolyn Vilter <br>
MSCAPP Data Viz <br>
Final Project <br>
Data Cleaning/Prep <br>

In [62]:
# Setup
import pandas as pd
import numpy as np

In [63]:
# Data
main = pd.read_csv("raw/main_data.csv")
predicted = pd.read_csv("raw/predicted_values.csv")
qualitative = pd.read_csv("raw/qual_data_selected.csv")
qual_binary = pd.read_csv("raw/qual_data_binary.csv")
dates = pd.read_csv("raw/significant_dates.csv")

### Line Chart: FEV1 over time

In [64]:
# Keep vars, drop NAs
line_data = main[["date", "fev1_over_pred_100"]]
line_data = line_data[line_data["fev1_over_pred_100"].notna()]
# Export
line_data.to_csv("clean/line_data.csv")

In [65]:
# Split out pre-treatment and treatment-to-plateau datasets for special subset charts
line_data['date'] = line_data['date'].astype('datetime64[ns]')
treatment_date = dates[dates["significance"] == "Began treatment"]["date"].iloc[0] # pulled from sig date data
plateau_start = "7/19/2019"
# Make sub datasets
line_pre_tre = line_data[(line_data["date"] < treatment_date)]
line_tre_plat = line_data[(line_data["date"] >= treatment_date) & (line_data["date"] <= plateau_start)]
# Export
line_pre_tre.to_csv("clean/line_pre_tre.csv")
line_tre_plat.to_csv("clean/line_tre_plat.csv")

### Bar Chart: Qual data recorded or not

In [66]:
# Format date var and group by week
qual_binary["date"] = pd.to_datetime(qual_binary["date"])
qual_binary['week'] = qual_binary.date.dt.to_period('W')
bar_data = qual_binary.groupby('week')['wrote_notes'].apply(lambda x: (x==1).sum()).reset_index(name='count_notes')
# Name the index and start at 1 (sorry) as in "week 1"
bar_data.index.name = 'week_num'
bar_data.index += 1
# Export
bar_data.to_csv("clean/bar_data.csv")

### Scatter: FEV1 vs. PEF

In [67]:
# Two vars, no NAs
scatter_data = main[["pef_over_pred_100", "fev1_over_pred_100"]]
scatter_data = scatter_data.dropna(axis=0, how="any")
# Export
scatter_data.to_csv("clean/scatter_data.csv")

### Histogram: PEF before and after treatment

In [68]:
hist_data = main[["date", "pef_over_pred_100"]]
hist_data = hist_data.dropna(axis=0, how="any")
# Format "date"
hist_data['date'] = hist_data['date'].astype('datetime64[ns]')
# Create before/after treatment distinction
hist_data["category"] = np.where(hist_data["date"] >= treatment_date, "after", "before")
hist_data = hist_data[["category", "pef_over_pred_100"]]

In [69]:
# Divide pef data into before and after treatment 
hist_before = hist_data[hist_data["category"] == "before"]
hist_after = hist_data[hist_data["category"] == "after"]
# Export
hist_before.to_json("clean/hist_before.json", orient="records")
hist_after.to_json("clean/hist_after.json", orient="records")